In [11]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Optional

In [9]:
# define the state structure
class BatsmanState(TypedDict):
    name: Optional[str]
    runs: int
    balls_faced: int
    fours: int
    sixes: int

    strike_rate: float
    bpb: float  # balls per boundary 
    boundary_percent: float
    summary: Optional[str]

In [7]:
def calculate_sr(state: BatsmanState) -> BatsmanState:
    state['strike_rate'] = (state['runs'] / state['balls_faced']) * 100 if state['balls_faced'] > 0 else 0
    return state

In [6]:
# calculate balls per boundary
def calculate_bpb(state: BatsmanState) -> BatsmanState:
    total_boundaries = state["fours"] + state["sixes"]
    state["bpb"] = (state["balls_faced"] / total_boundaries) if total_boundaries > 0 else float('inf')
    return state

In [8]:
# calcualte boundary percentage
def calculate_boundary_percent(state: BatsmanState) -> BatsmanState:
    total_boundaries = state["fours"] + state["sixes"]
    state["boundary_percent"] = (total_boundaries / state["balls_faced"]) * 100 if state["balls_faced"] > 0 else 0
    return state

In [10]:
# calculate final output
def final_output(state: BatsmanState) -> BatsmanState:
    summary = f"name: {state.get('name', 'Unknown')}, runs: {state['runs']}, balls faced: {state['balls_faced']}, strike rate: {state['strike_rate']:.2f}, bpb: {state['bpb']:.2f}, boundary %: {state['boundary_percent']:.2f}"
    state["summary"] = summary
    return state

In [ ]:
# define graph
graph = StateGraph(BatsmanState)

# adding nodes
graph.add_node("calculate_sr", calculate_sr)
graph.add_node("calculate_bpb", calculate_bpb)
graph.add_node("calculate_boundary_percent", calculate_boundary_percent)
graph.add_node("final_output", final_output)

# defining edges for parallel processing
graph.add_edge(START, "calculate_sr")
graph.add_edge(START, "calculate_bpb")
graph.add_edge(START, "calculate_boundary_percent")

graph.add_edge("calculate_sr", "final_output")
graph.add_edge("calculate_bpb", "final_output")
graph.add_edge("calculate_boundary_percent", "final_output")

graph.add_edge("final_output", END)